# LLM 및 임베딩 모델 파인튜닝 실습 (Colab)

1. **LLM 파인튜닝** (Unsloth + Llama-3.1)
   - 대규모 언어 모델을 특정 도메인(ETF 정보)에 맞게 학습
   - LoRA 기법을 사용한 효율적인 파인튜닝

2. **임베딩 모델 파인튜닝** (Sentence Transformers)
   - 텍스트를 벡터로 변환하는 모델 학습
   - 의미적으로 유사한 문장을 가깝게 배치

---

## 1. 환경 설정

- **GPU (Graphics Processing Unit)**:
    - 원래 그래픽 처리용으로 설계됨
    - 수천 개의 작은 코어가 병렬로 작동
    - 딥러닝의 행렬 연산에 최적화됨
    - CPU 대비 10~100배 빠른 학습 속도

- **CUDA**:
    - NVIDIA GPU를 프로그래밍하기 위한 플랫폼
    - PyTorch, TensorFlow 등이 CUDA를 사용
    - GPU 메모리(VRAM)가 클수록 큰 모델 학습 가능

In [ ]:
# GPU 확인
# 현재 시스템에 사용 가능한 GPU 정보를 확인합니다.
# - GPU 모델명, 메모리 사용량, 온도 등을 표시
!nvidia-smi

### 필수 패키지 설치

1. **unsloth**: LLM 파인튜닝을 최적화한 라이브러리
   - 메모리 사용량 감소 (최대 80%)
   - 학습 속도 향상 (최대 2배)
   - LoRA, QLoRA 등 효율적인 파인튜닝 지원

2. **sentence-transformers**: 임베딩 모델 라이브러리
   - 텍스트를 고정 길이 벡터로 변환
   - 의미적 유사도 계산에 사용

3. **pandas**: 데이터 처리 및 분석

4. **python-dotenv**: 환경 변수 관리 (.env 파일)

In [ ]:
# Colab 환경에서 Unsloth 및 필수 패키지 설치
%%capture
import os, re
import torch

# Unsloth 설치 (Colab 전용)
v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
!pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
!pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

# 기타 패키지 설치
!pip install pandas python-dotenv ipykernel
!pip install -U "sentence-transformers>=3.0"

### Hugging Face 인증

- **Hugging Face**: 머신러닝 모델과 데이터셋을 공유하는 플랫폼
    - **Hub**: 모델, 데이터셋, 데모를 호스팅
    - **Token**: API 접근을 위한 인증 키
    - **공개 vs 비공개**: 모델을 공개하거나 개인용으로 설정 가능

- **Token 생성 방법**:
    1. [Hugging Face](https://huggingface.co) 접속
    2. Settings → Access Tokens
    3. New Token → Write 권한 선택
    4. 생성된 토큰 복사

In [ ]:
# 환경 변수 설정
import os
from dotenv import load_dotenv
from huggingface_hub import login
from google.colab import userdata

# Colab Secrets에서 토큰 로드 시도
try:
    os.environ["HUGGINGFACE_TOKEN"] = userdata.get('HUGGINGFACE_TOKEN')
except:
    pass

# 직접 입력 (Secrets에 없는 경우)
if "HUGGINGFACE_TOKEN" not in os.environ or not os.environ["HUGGINGFACE_TOKEN"]:
    from getpass import getpass
    # getpass: 입력 시 화면에 토큰이 표시되지 않음 (보안)
    os.environ["HUGGINGFACE_TOKEN"] = getpass("Hugging Face Token: ")

# Hugging Face에 로그인
# add_to_git_credential=True: git 자격증명에 추가하여 다음에 자동 로그인
login(token=os.environ["HUGGINGFACE_TOKEN"], add_to_git_credential=True)

### 기본 라이브러리 임포트 및 환경 확인

In [ ]:
# 기본 import
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
import torch
import time
import warnings
warnings.filterwarnings('ignore')  # 경고 메시지 숨기기

# PyTorch 및 CUDA 정보 출력
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    # Compute Capability: GPU의 기능 수준 (높을수록 최신 기능 지원)
    print(f"CUDA Capability: {torch.cuda.get_device_capability(0)}")
    # VRAM: GPU 메모리 (모델 크기와 배치 크기를 결정하는 중요한 요소)
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

---

## 2. 데이터 준비

- **LLM 파인튜닝 데이터셋 (Alpaca 형식)**:
  ```json
  {
    "instruction": "이 ETF의 기본 정보를 제공해주세요.",
    "input": "TIGER 200",
    "output": "TIGER 200은 KOSPI 200 지수를 추종하는..."
  }
  ```

- **임베딩 파인튜닝 데이터셋 (Sentence Pair)**:
  ```json
  {
    "sentence1": "TIGER 200은 KOSPI 200을 추종합니다",
    "sentence2": "이 ETF는 한국 주요 200개 기업에 투자합니다"
  }
  ```
  - 의미적으로 유사한 문장 쌍을 학습
  - 모델이 같은 의미를 가진 문장을 가깝게 임베딩하도록 학습

In [ ]:
# 작업 디렉토리 설정 (Colab의 임시 스토리지)
WORKSPACE = "/content"
DATA_DIR = f"{WORKSPACE}/etf_data"
MODEL_DIR = f"{WORKSPACE}/models"

import os
# exist_ok=True: 디렉토리가 이미 존재해도 에러 발생하지 않음
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"작업 디렉토리: {WORKSPACE}")
print(f"데이터: {DATA_DIR}")
print(f"모델: {MODEL_DIR}")

### Hugging Face 데이터셋 로드

- **Train Set (학습 데이터)**:
    - 모델이 패턴을 학습하는 데 사용
    - 일반적으로 전체 데이터의 80-90%

- **Test Set (테스트 데이터)**:
    - 모델의 일반화 성능을 평가
    - 학습에 사용되지 않은 새로운 데이터
    - 과적합(overfitting) 여부 확인

In [ ]:
# HF 데이터셋 로드
from datasets import load_dataset

# LLM 데이터셋
username = '********'   # 각자 본인의 Huggingface ID로 설정

# 예시 데이터셋이 없다면 redwiggler/alpaca-cleaned 등을 사용할 수 있습니다.
# alpaca_dataset = load_dataset("yahma/alpaca-cleaned")
alpaca_dataset = load_dataset(f"{username}/etf-alpaca-llm-v1")

print(f"✅ LLM 데이터셋 로드 완료!")
print(f"   Train: {len(alpaca_dataset['train'])}개")
print(f"   Test: {len(alpaca_dataset['test'])}개")

# 임베딩 데이터셋
embedding_dataset = load_dataset(f"{username}/etf-embedding-v1")

print(f"✅ 임베딩 데이터셋 로드 완료!")
print(f"   Train: {len(embedding_dataset['train'])}개")
print(f"   Test: {len(embedding_dataset['test'])}개")

# 샘플 확인
print("\nLLM 샘플:")
print(alpaca_dataset['train'][0])

print("\n임베딩 샘플:")
print(embedding_dataset['train'][0])

---

## 3. LLM 파인튜닝

- **사전 학습(Pre-training)**:
    - 대규모 텍스트 데이터로 모델을 학습
    - 일반적인 언어 패턴과 지식 습득
    - 예: GPT, Llama는 인터넷의 방대한 텍스트로 학습됨

- **파인튜닝(Fine-tuning)**:
    - 사전 학습된 모델을 특정 작업/도메인에 맞게 추가 학습
    - 적은 데이터로도 효과적
    - 예: ETF 정보에 특화된 모델 만들기

- **LoRA** (Low-Rank Adaptation)

    - **전통적 파인튜닝의 문제**:
        - 모든 파라미터 업데이트 → 메모리 많이 필요
        - 학습 시간 오래 걸림
        - 모델 크기만큼의 저장 공간 필요

    - **LoRA의 해결책**:
        - 원본 모델은 고정(freeze)
        - 작은 어댑터(adapter) 레이어만 학습
        - 메모리 사용량 80% 감소
        - 학습 속도 2-3배 향상
        - 어댑터만 저장 (수십 MB vs 수십 GB)

    - **LoRA 파라미터**:
        - `r`: rank (낮을수록 파라미터 적음, 일반적으로 8-64)
        - `lora_alpha`: 학습률 스케일링
        - `target_modules`: 어떤 레이어에 LoRA 적용할지

- **Quantization (양자화)**
    - 모델 가중치를 낮은 정밀도로 저장
    - FP32 (32비트) → INT4/INT8 (4비트/8비트)
    - 메모리 사용량 1/4 ~ 1/8로 감소
    - 성능 손실 최소 (1-2%)
    - 더 큰 모델을 같은 GPU에서 학습 가능

In [ ]:
from unsloth import FastLanguageModel

# 모델 선택
MODEL_NAME = "unsloth/Meta-Llama-3.1-8B"  # Llama 3.1 8B 모델
MAX_SEQ_LENGTH = 4096  # 최대 토큰 길이 (Colab T4에서는 메모리 주의)

# 모델 로드
# load_in_4bit: 4비트 양자화 사용 여부 (메모리 절약)
# load_in_8bit: 8비트 양자화 사용 여부 (4비트보다 정확, 메모리 2배)
# full_finetuning: 전체 파라미터 학습 vs LoRA (False 권장)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = True,  # Colab T4에서는 4비트 필수
    load_in_8bit = False,
    full_finetuning = False,  # LoRA 사용
)

### 프롬프트 템플릿 및 데이터 전처리

- **프롬프트 엔지니어링**:
    - 모델에게 작업을 명확하게 지시하는 방법
    - 일관된 형식으로 입력을 구조화

- **Alpaca 형식**:

    ```
    ### Instruction:
    [무엇을 해야 하는지]

    ### Input:
    [구체적인 입력 데이터]

    ### Response:
    [모델의 출력]
    ```

- **EOS Token (End of Sequence)**:
    - 문장/대화의 끝을 표시하는 특수 토큰
    - 추가하지 않으면 모델이 계속 생성함
    - 학습 시 반드시 포함해야 함

In [ ]:
# 프롬프트 템플릿 정의
alpaca_prompt = """Below is an instruction that describes a task, paired with an input. Write a response.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # 문장 종료 토큰 (필수!)

def formatting_prompts_func(examples):
    """
    데이터를 프롬프트 형식으로 변환하는 함수

    Args:
        examples: instruction, input, output을 포함한 배치 데이터

    Returns:
        프롬프트 형식의 텍스트 리스트
    """
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        # EOS_TOKEN을 반드시 추가해야 생성이 무한히 계속되지 않음
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return { "text" : texts }

# 데이터셋 전처리
# batched=True: 배치 단위로 처리하여 속도 향상
train_data = alpaca_dataset['train'].map(formatting_prompts_func, batched=True)
test_data  = alpaca_dataset['test'].map(formatting_prompts_func, batched=True)

# 변환된 데이터 확인
print(train_data[0]['text'])

### 학습 전 모델 테스트

- **Inference(추론) 모드**:
    - 학습이 아닌 예측/생성 모드
    - 그래디언트 계산 비활성화 (메모리 절약)
    - 드롭아웃 등 학습용 기능 비활성화

- **생성 파라미터**:
    - `max_new_tokens`: 생성할 최대 토큰 수
    - `temperature`: 다양성 조절 (높을수록 창의적, 낮을수록 결정적)
    - `top_p`: nucleus sampling (상위 p% 확률 토큰만 고려)
    - `repetition_penalty`: 반복 방지 (>1.0이면 반복 억제)

In [ ]:
# 모델을 추론 모드로 전환
FastLanguageModel.for_inference(model)

def test_model(instruction, input_text=""):
    """
    모델 테스트 함수

    Args:
        instruction: 작업 지시
        input_text: 입력 데이터

    Returns:
        모델의 응답
    """
    # 프롬프트 생성 (Response 부분은 비워둠)
    prompt = alpaca_prompt.format(instruction, input_text, "")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,       # 최대 512 토큰 생성
        temperature=0.7,          # 적당한 다양성
        top_p=0.9,                # nucleus sampling
        repetition_penalty=1.1    # 반복 억제
    )

    # 생성된 텍스트 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Response 부분만 추출
    return response.split("### Response:")[-1].strip()

# 여러 테스트 케이스
test_cases = [
    ("다음 ETF의 기본 정보를 제공해주세요.", "ACE 레버리지"),
    ("이 ETF의 투자 정보를 알려주세요.", "TIGER KOFR"),
    ("어떤 ETF를 추천하나요?", "안정적인 투자를 원합니다")
]

print("\n" + "="*60)
print("학습 전 모델 테스트")
print("="*60)
for inst, inp in test_cases:
    print(f"\n[질문] {inst}")
    print(f"[입력] {inp}")
    print(f"[답변] {test_model(inst, inp)}")
    print("-"*60)

### LoRA 어댑터 추가

- **r (rank)**:
    - LoRA 행렬의 차원
    - 낮을수록 파라미터 수 감소
    - 권장값: 8 (간단), 16-32 (일반), 64-128 (복잡)

- **target_modules**:
    - LoRA를 적용할 레이어 선택
    - `q_proj, k_proj, v_proj`: Attention의 Query, Key, Value
    - `o_proj`: Attention 출력
    - `gate_proj, up_proj, down_proj`: Feed-Forward 네트워크

- **lora_alpha**:
    - LoRA 업데이트의 스케일링 계수
    - 일반적으로 r과 같은 값 사용

- **lora_dropout**:
    - 과적합 방지를 위한 드롭아웃
    - 0 = 최적화됨 (Unsloth 권장)

- **use_gradient_checkpointing**:
    - 메모리 절약 기법
    - "unsloth" = Unsloth 최적화 버전 (30% 메모리 절약)
    - 배치 크기를 2배 늘릴 수 있음

In [ ]:
# 학습 모드로 전환
FastLanguageModel.for_training(model)

# LoRA 어댑터 추가
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,  # LoRA rank (파라미터 수 결정)
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention 레이어
        "gate_proj", "up_proj", "down_proj",     # FFN 레이어
    ],
    lora_alpha = 8,  # 스케일링 계수
    lora_dropout = 0,  # 드롭아웃 (0 = 최적화)
    bias = "none",     # 바이어스 학습 안 함 (최적화)
    use_gradient_checkpointing = "unsloth",  # 메모리 절약 (30%)
    random_state = 3407,  # 재현성을 위한 시드
    use_rslora = False,   # Rank-Stabilized LoRA
    loftq_config = None,  # LoftQ 양자화
)

### 학습 파라미터 설정 및 시작

- **배치(Batch)**:
    - 한 번에 처리하는 샘플 수
    - `per_device_train_batch_size`: GPU당 배치 크기
    - 클수록 학습 안정적, 메모리 많이 필요

- **Gradient Accumulation**:
    - 여러 미니배치의 그래디언트를 누적
    - 실제 배치 크기 = batch_size × accumulation_steps
    - 메모리 부족 시 유용

- **에포크(Epoch)**:
    - 전체 데이터를 한 번 학습하는 단위
    - `num_train_epochs`: 총 에포크 수
    - 또는 `max_steps`: 총 스텝 수로 제한 가능

- **학습률(Learning Rate)**:
    - 파라미터 업데이트 크기
    - 너무 크면 불안정, 너무 작으면 느림
    - LoRA: 2e-4 ~ 5e-4 권장

- **Warmup**:
    - 초반에 학습률을 서서히 증가
    - 학습 안정성 향상

- **Learning Rate Scheduler**:
    - `cosine`: 코사인 함수 형태로 감소 (일반적)
    - `linear`: 선형으로 감소
    - `constant`: 고정

- **Mixed Precision (FP16/BF16)**:
    - FP32 대신 FP16/BF16 사용
    - 메모리 절반, 속도 2배
    - BF16이 더 안정적 (Ampere GPU 이상)

- **옵티마이저**:
    - `adamw_8bit`: 8비트 AdamW (메모리 절약)
    - `adamw_torch`: 표준 AdamW
    - `sgd`: Stochastic Gradient Descent

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# 학습 설정
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="text",  # 텍스트가 저장된 필드명
    max_seq_length=MAX_SEQ_LENGTH,
    args=TrainingArguments(
        # 배치 크기 (GPU 메모리에 따라 조정)
        per_device_train_batch_size=2,   # Colab T4: 2 권장
        gradient_accumulation_steps=4,   # 실제 배치 = 2 × 4 = 8

        # 학습 기간
        warmup_steps=10,        # 초반 워밍업 스텝
        # num_train_epochs=2,     # 총 2 에포크 학습
        max_steps=60,         # 또는 스텝 수로 제한 (테스트용)

        # 최적화
        learning_rate=2e-4,            # 학습률
        optim="adamw_8bit",            # 8비트 AdamW
        weight_decay=0.01,             # 가중치 감쇠 (정규화)
        lr_scheduler_type="cosine",    # 코사인 스케줄러

        # Mixed Precision
        fp16=not torch.cuda.is_bf16_supported(),  # BF16 지원 안 되면 FP16
        bf16=torch.cuda.is_bf16_supported(),      # BF16 지원되면 사용

        # 로깅 및 저장
        logging_steps=1,              # 매 스텝마다 로그
        save_strategy="epoch",        # 에포크마다 저장
        eval_strategy="epoch",        # 에포크마다 평가
        load_best_model_at_end=True,  # 최고 성능 모델 로드

        # 출력 디렉토리
        output_dir=f"{MODEL_DIR}/llm_outputs",
        seed=3407,           # 재현성
        report_to="none",    # wandb 등 비활성화
    ),
)

print("🚀 학습 시작...")
import time
start_time = time.time()

# 학습 실행
trainer.train()

elapsed = time.time() - start_time
print(f"✅ 학습 완료! (소요 시간: {elapsed/60:.2f}분)")

### 학습 후 모델 테스트

In [ ]:
# 모델을 추론 모드로 전환
FastLanguageModel.for_inference(model)

def test_model(instruction, input_text=""):
    prompt = alpaca_prompt.format(instruction, input_text, "")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Response:")[-1].strip()

# 여러 테스트 케이스
test_cases = [
    ("다음 ETF의 기본 정보를 제공해주세요.", "ACE 레버리지"),
    ("이 ETF의 투자 정보를 알려주세요.", "TIGER KOFR"),
    ("어떤 ETF를 추천하나요?", "안정적인 투자를 원합니다")
]

print("\n" + "="*60)
print("학습 후 모델 테스트")
print("="*60)
for inst, inp in test_cases:
    print(f"\n[질문] {inst}")
    print(f"[입력] {inp}")
    print(f"[답변] {test_model(inst, inp)}")
    print("-"*60)

### 모델 저장

- **1. LoRA 어댑터만 저장**:
    - 가장 작은 크기 (수십 MB)
    - 원본 모델과 함께 로드 필요
    - Hugging Face에 업로드 권장

- **2. 병합된 16-bit 모델**:
    - 원본 + LoRA를 하나로 병합
    - 독립적으로 사용 가능
    - 크기 증가 (수 GB)

- **3. GGUF 형식**:
    - llama.cpp, Ollama 등에서 사용
    - 다양한 양자화 수준 지원
    - `q4_k_m`: 4비트 (빠르고 작음)
    - `q8_0`: 8비트 (더 정확)
    - CPU에서도 실행 가능

In [ ]:
# 모델 저장 (여러 형식)
save_dir = f"{MODEL_DIR}/etf_llama_model"

# 1. LoRA 어댑터만 저장 (가장 작음, 권장)
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"✅ LoRA 어댑터 저장: {save_dir}")

# 2. 병합된 16-bit 모델 저장 (배포용)
# model.save_pretrained_merged(
#     f"{save_dir}_merged_16bit",
#     tokenizer,
#     save_method="merged_16bit"
# )
# print(f"✅ 병합 모델 저장: {save_dir}_merged_16bit")

# 3. GGUF 형식 저장 (Ollama, llama.cpp용)
# model.save_pretrained_gguf(
#     f"{save_dir}_gguf",
#     tokenizer,
#     quantization_method=["q4_k_m", "q8_0"]  # 4비트, 8비트 버전
# )
# print(f"✅ GGUF 저장: {save_dir}_gguf")

### Hugging Face Hub에 업로드

- **Hugging Face Hub의 장점**:
    - 클라우드 저장소 (무료)
    - 버전 관리 (Git 기반)
    - 어디서든 다운로드 가능
    - 팀 협업 가능
    - 공개/비공개 설정

- **리포지토리 명명 규칙**:
    ```
    username/model-name
    예: john/llama-7B-etf-finetuned
    ```

In [ ]:
# Hugging Face 업로드
repo_name = f"{username}/llama-7B-etf-finetuned"

# LoRA 어댑터 업로드
model.push_to_hub(
    repo_name,
    token=os.environ["HUGGINGFACE_TOKEN"],
    private=True  # 비공개 리포지토리
)
tokenizer.push_to_hub(
    repo_name,
    token=os.environ["HUGGINGFACE_TOKEN"],
    private=True
)
print(f"✅ 모델 업로드 완료: https://huggingface.co/{repo_name}")

---

## 4. 임베딩(Embedding) 모델 파인튜닝

- **임베딩의 정의**:
    - 텍스트를 고정 길이의 숫자 벡터로 변환
    - 의미적으로 유사한 텍스트 → 가까운 벡터
    - 예: "강아지"와 "개" → 유사한 벡터

- **임베딩의 활용**:
    1. **유사도 검색**: 가장 관련 있는 문서 찾기
    2. **클러스터링**: 비슷한 텍스트 그룹화
    3. **분류**: 텍스트 카테고리 예측
    4. **RAG**: 관련 문서를 검색하여 LLM에 제공

- **Sentence-BERT (SBERT)**:
    - BERT를 문장 임베딩에 최적화
    - Siamese 네트워크 구조
    - 빠른 속도 (BERT 대비 100배)

- **BGE-M3 모델**:
    - **B**AIRU **G**eneral **E**mbedding
    - **M**ulti-lingual (다국어 지원)
    - **M**ulti-granularity (다양한 길이)
    - **M**ulti-functionality (다양한 태스크)
    - 1024차원 임베딩

In [ ]:
from sentence_transformers import SentenceTransformer

# 모델 로드
# BGE-M3: 다국어 지원, 고성능 임베딩 모델
emb_model = SentenceTransformer("BAAI/bge-m3")
print(f"임베딩 차원: {emb_model.get_sentence_embedding_dimension()}")
print(f"최대 시퀀스 길이: {emb_model.max_seq_length}")

### 손실 함수 및 평가자 설정

- **Multiple Negatives Ranking Loss**: 대조 학습(Contrastive Learning)
    - Positive Pair: 의미적으로 유사한 문장 쌍
    - Negative Samples: 배치 내 다른 문장들
    - 목표: Positive는 가깝게, Negative는 멀게
    - 배치 크기가 클수록 효과적 (더 많은 negative samples)

- **Matryoshka Embeddings**: 마트료시카 인형 원리
    - 하나의 모델이 여러 차원의 임베딩 생성
    - 예: 768차원, 512차원, 256차원, 128차원, 64차원
    - 사용 시 필요한 차원만 선택
    - 저차원: 빠르고 메모리 절약
    - 고차원: 더 정확

- **장점**:
    - 유연성: 상황에 맞게 차원 선택
    - 효율성: 하나의 모델로 다양한 용도
    - 성능 저하 최소화

- **Spearman 상관계수**: 임베딩 평가
    - 순위 기반 상관관계 측정
    - -1 ~ 1 범위
    - 1에 가까울수록 좋음
    - 인간의 유사도 판단과 모델 예측 비교

In [ ]:
from sentence_transformers.losses import MultipleNegativesRankingLoss, MatryoshkaLoss
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments

# 손실 함수 설정
# 1. 기본 손실: Multiple Negatives Ranking Loss
base_loss = MultipleNegativesRankingLoss(emb_model)

# 2. Matryoshka Loss로 래핑 (다양한 차원 지원)
train_loss = MatryoshkaLoss(
    emb_model,
    base_loss,
    matryoshka_dims=[1024, 768, 512, 256, 128, 64]  # 지원할 차원들
)

# 평가자 설정
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=embedding_dataset["test"]["sentence1"],
    sentences2=embedding_dataset["test"]["sentence2"],
    scores=embedding_dataset["test"]["label"],
    name="etf-eval"
)

# 학습 전 성능 평가
score_before = evaluator(emb_model)
print(score_before)

### 임베딩 모델 학습

- LLM보다 빠름 (작은 모델)
- 배치 크기를 크게 설정 가능
- 대조 학습에서는 큰 배치가 유리
- 학습률: 2e-5 권장 (LLM보다 낮음)

In [ ]:
# 임베딩 학습
emb_trainer = SentenceTransformerTrainer(
    model=emb_model,
    train_dataset=embedding_dataset['train'],
    eval_dataset=embedding_dataset['test'],
    loss=train_loss,
    evaluator=evaluator,
    args=SentenceTransformerTrainingArguments(
        output_dir=f"{MODEL_DIR}/etf_embedding",

        # 학습 설정
        num_train_epochs=1,  # 총 에포크
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,

        # 최적화
        warmup_ratio=0.1,        # 전체의 10%를 워밍업
        learning_rate=2e-5,      # 임베딩 모델은 낮은 학습률
        fp16=True,               # Mixed Precision

        # 평가 및 저장
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=3,            # 최근 3개 체크포인트만 유지
        load_best_model_at_end=True,   # 최고 성능 모델 로드

        # 로깅
        logging_steps=10,
        report_to="none",
    )
)

print("🚀 임베딩 학습 시작...")
start_time = time.time()

emb_trainer.train()

elapsed = time.time() - start_time
print(f"✅ 임베딩 학습 완료! (소요 시간: {elapsed/60:.2f}분)")

# 학습 후 성능 평가
score_after = evaluator(emb_model)
print(score_after)

### 임베딩 모델 저장 및 업로드

In [ ]:
# 임베딩 모델 저장
emb_save_dir = f"{MODEL_DIR}/etf_embedding_final"
emb_model.save(emb_save_dir)
print(f"✅ 로컬 저장: {emb_save_dir}")

# Hugging Face Hub에 업로드
emb_repo = f"{username}/etf-embedding-bge-m3"
emb_model.push_to_hub(
    emb_repo,
    token=os.environ["HUGGINGFACE_TOKEN"],
    private=True
)
print(f"✅ HF Hub 업로드: https://huggingface.co/{emb_repo}")

---

## 5. 임베딩 모델 테스트

- **코사인 유사도(Cosine Similarity)**:
    - 두 벡터 사이의 각도 측정
    - -1 ~ 1 범위
    - 1: 완전히 동일한 방향
    - 0: 직교 (관련 없음)
    - -1: 완전히 반대 방향

- **임베딩에서의 활용**:
    - 문장 간 유사도 측정
    - 검색: 쿼리와 문서 유사도
    - 추천: 아이템 간 유사도

In [ ]:
from sentence_transformers.util import cos_sim

# 테스트 문장들
test_sentences = [
    "TIGER 200은 KOSPI 200을 추종하는 ETF입니다.",
    "이 ETF는 한국 주요 200개 기업에 투자합니다.",
    "레버리지 ETF는 고위험 고수익 상품입니다.",
    "오늘 날씨가 참 좋네요."
]

# 임베딩 생성
embeddings = emb_model.encode(test_sentences, convert_to_tensor=True)

# 유사도 행렬 계산
similarity_matrix = cos_sim(embeddings, embeddings)

print("\n📊 문장 간 유사도 행렬:\n")
print("\t" + "\t".join([f"문장{i+1}" for i in range(len(test_sentences))]))
for i, row in enumerate(similarity_matrix):
    print(f"문장{i+1}\t" + "\t".join([f"{val:.3f}" for val in row]))